In [1]:
import graph as gr
import numpy as np
import networkx as nx

import torch
torch.manual_seed(12121998)
import torch.nn.functional as F

from torch.nn import Linear, ReLU
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
from torch_geometric.nn.models import basic_gnn as torch_models

import matplotlib.pyplot as plt
import matplotlib.tri as tri

c:\ProgramData\Anaconda3\envs\my-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Creating Dataset

In [2]:
# Macros

NORMALIZE_TARGET_FLAG = True # normalize output!
if NORMALIZE_TARGET_FLAG:
    norm_inter = [0, 1] # Interval where to normalize data

COORDINATES_FEATURE_FLAG = True # If true also position is used as feature!

In [3]:
# FILENAMES AND FILEPATH

# mesh name
mesh_filename = "simple_laplacian_mesh"

## target filepaths
target_filepaths = ["../mesh/simple_laplacian_u.txt"] # you can add other targets

## feature filepaths 
feature_filepaths = ["../mesh/simple_laplacian_f.txt"]

In [4]:
folder = gr.import_mesh(mesh_filename)

loaded_graph = gr.build_graph(folder)

loaded_adj = gr.build_adjacency(loaded_graph, folder)

loaded_targets = [torch.tensor(0)] * len(target_filepaths)
for i, data in enumerate(target_filepaths):
    loaded_targets[i] = torch.tensor(gr.import_features(data)).double()

loaded_features = [torch.tensor(0)] * len(feature_filepaths)
for i, data in enumerate(feature_filepaths):
    loaded_features[i] = torch.tensor(gr.import_features(data)).double()


if NORMALIZE_TARGET_FLAG:
    u_max = [None] * len(loaded_targets)
    u_min = [None] * len(loaded_targets)
    for i, data in enumerate(loaded_targets):
        u_max[i] = data.max()
        u_min[i] = data.min()
        loaded_targets[i] = (norm_inter[1] - norm_inter[0]) * (data - u_min[i]) / (u_max[i] - u_min[i]) + norm_inter[0]  


TOT_SAMPLES = loaded_targets[0].shape[1]

In [5]:
## Creating torch DATASET (see torch_geometric.data)

# will be used to create the complete dataset
data_list = [torch.tensor(0)] * TOT_SAMPLES

# INITIALIZING COMMON PROPERTIES TO ALL GRAPHS

# Graph connectivity (# use t().contiguous() ...)
edge_index = torch.tensor([ [i,j] for i in range(loaded_adj.shape[0]) for j in range(loaded_adj.shape[1]) if loaded_adj[i,j]>0 ])

# Node position matrix ( ACTUALLY, IT SHOULD NOT BE NEEDED SINCE IT'S CONSTANT ... )
# check whether pos is used in the training or not. In the second case it's useless for dynamic meshes
pos = torch.hstack( (
                        torch.unsqueeze(torch.tensor(list(nx.get_node_attributes(loaded_graph, 'x').values())), 1 ),
                        torch.unsqueeze(torch.tensor(list(nx.get_node_attributes(loaded_graph, 'y').values())), 1 ) 
                    )
                  ).double()

# INITIALIZING THE DIFFERENT GRAPHS

for t in range(TOT_SAMPLES):
    
    ## Node feature matrix
    
    # REMOVED TYPE OF NODE FEATURE!
    # first feature (type of node)
    #x = torch.unsqueeze(torch.tensor(list(nx.get_node_attributes(loaded_graph, 'n').values())), 1).double()
    
    x = loaded_features[0][:, t][:, None]

    # other features (forcing f values)
    [x := torch.hstack((x, loaded_features[j][:, t][:, None])) for j in range(1,len(loaded_features))]
    
    if COORDINATES_FEATURE_FLAG:
        [x := torch.hstack((x, pos))]
    
    
    ## Node-level ground-truth labels (heat)
    
    # first feature (pression or velocity or ...)
    y = loaded_targets[0][:, t][:, None]
    
    # other features (pression, velocities, ...)
    [y := torch.hstack((y, loaded_targets[j][:, t][:, None])) for j in range(1,len(loaded_targets))]

    data_list[t] = Data(x=x, edge_index=edge_index.t().contiguous(), y=y, pos=pos)

In [6]:
dataloader = DataLoader(data_list, batch_size = 1, shuffle = True)

In [7]:
len(dataloader)

1000

# Model

In [8]:
class GCN(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataloader.dataset[0].num_node_features, 16)
        self.conv2 = GCNConv(16, 16)
        self.fc1 = Linear(16, dataloader.dataset[0].num_nodes)

    def forward(self, x, edge_index):
     
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.tanh(self.fc1(x))
        
        return x.T

model = GCN().double()

In [21]:
conv1 = GCNConv(dataloader.dataset[0].num_node_features, dataloader.dataset[0].num_node_features)
#out = conv1(dataloader.dataset[0].x, edge_index)

dataloader.dataset[0].x.size()

torch.Size([1089, 1])

In [9]:
learning_rate = 1e-3

loss_fn = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    loss = 0
    for batch, data in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(data.x, data.edge_index)
        loss = loss_fn(pred, data.y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss += loss.item()
    print("loss: {}".format(loss))

In [11]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------


c:\ProgramData\Anaconda3\envs\my-env\lib\site-packages\torch\nn\functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
c:\ProgramData\Anaconda3\envs\my-env\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([1089, 1])) that is different to the input size (torch.Size([1089, 1089])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 

In [ ]:
pred = (model(dataloader.dataset[0]['x'], dataloader.dataset[0]['edge_index'])- norm_inter[0]) / (norm_inter[1] - norm_inter[0]) * (u_max[0] - u_min[0]) + u_min[0]
pred = torch.tensor(pred[:, 0])
torch.set_printoptions(profile = "full")
pred

In [ ]:
true = ((data_list[0].y) - norm_inter[0]) / (norm_inter[1] - norm_inter[0]) * (u_max[0] - u_min[0]) + u_min[0]
true= torch.tensor(true[:, 0])
print(true)

In [ ]:
error = abs(true - pred)
error.mean()

In [ ]:
# plots a finite element mesh
def plot_fem_mesh(nodes_x, nodes_y, elements):
    for element in elements:
        x = [nodes_x[element[i]] for i in range(len(element))]
        y = [nodes_y[element[i]] for i in range(len(element))]
        plt.fill(x, y, edgecolor='black', fill=False, linewidth='0.5')

# FEM data
nodes_x = dataloader.dataset[0]['pos'][:,0]
nodes_y = dataloader.dataset[0]['pos'][:,1]
nodal_values = pred # loaded_features[0][:, 0]
elements = np.load(folder+"/triangles.npy")
triangulation = tri.Triangulation(nodes_x, nodes_y, elements)
print(nodes_x.size(), nodes_y.size(), nodal_values.size())

# plot the contours
plt.tricontourf(triangulation,  nodal_values)

# plot the finite element mesh
plot_fem_mesh(nodes_x, nodes_y, elements)

# show
plt.colorbar()
plt.axis('equal')
plt.show()

In [ ]:
# plots a finite element mesh
def plot_fem_mesh(nodes_x, nodes_y, elements):
    for element in elements:
        x = [nodes_x[element[i]] for i in range(len(element))]
        y = [nodes_y[element[i]] for i in range(len(element))]
        plt.fill(x, y, edgecolor='black', fill=False, linewidth='0.5')

# FEM data
nodes_x = dataloader.dataset[0]['pos'][:,0]
nodes_y = dataloader.dataset[0]['pos'][:,1]
nodal_values = true # loaded_features[0][:, 0]
elements = np.load(folder+"/triangles.npy")
triangulation = tri.Triangulation(nodes_x, nodes_y, elements)
print(nodes_x.size(), nodes_y.size(), nodal_values.size())

# plot the contours
plt.tricontourf(triangulation,  nodal_values)

# plot the finite element mesh
plot_fem_mesh(nodes_x, nodes_y, elements)

# show
plt.colorbar()
plt.axis('equal')
plt.show()